**The objective of this notebook is to collate the codes for cleaning below data:**
1. Bond yields (F)
2. Ineterest rate (F)
3. Population (K)
4. Construction (K)
5. Weekly income (A)
6. Household size (A)

(please add features to the list if there's any additional ones)

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### 1. Bond Yields ###
Please paste cleaning codes below

### 2. Interest Rate ###
Please paste cleaning codes below

### 3. Population / Age bands 
Please paste cleaning codes below

#### Household count
Options:
 - Use 2016, 2021, and delta
 - Use 2016 and 2021 data only
 - Use delta only

In [5]:
popfile = "Files/Population/2019 NSW Population Projections ASGS 2019 LGA.xlsx"
df_hhold_wide = pd.read_excel(popfile,sheet_name='LGA Household Totals',header=6,usecols="A:C",skipfooter=3)
df_hhold_wide.columns=['LGA','hhold_count_2016','hhold_count_2021']
df_hhold_wide['LGA'] = df_hhold_wide.LGA.str.split('(').str.get(0)

df_hhold_wide['hhold_count_delta'] = df_hhold_wide.hhold_count_2021 - df_hhold_wide.hhold_count_2016

df_hhold_wide.head()

,LGA,hhold_count_2016,hhold_count_2021,hhold_count_delta
0,Albury,21940,23227,1287
1,Armidale Regional,11755,13041,1286
2,Ballina,18178,19080,902
3,Balranald,963,1015,52
4,Bathurst Regional,16105,17351,1246


#### Population movement in 5 year period


In [7]:
df_move = pd.read_excel(popfile,sheet_name='LGA population accounts', header=5, skipfooter=3, usecols="A:C")
df_move.columns=['LGA','pop_move','2016-2021']

df_move_melt = pd.melt(df_move,id_vars=['LGA','pop_move'], value_vars=['2016-2021'], var_name='year')
df_move_pivot = df_move_melt.pivot(index=['LGA','year'], columns='pop_move', values='value').reset_index()
df_move_pivot['LGA'] = df_move_pivot.LGA.str.split('(').str.get(0)
df_move_pivot['pop_delta'] = df_move_pivot['Population at End of Period'] - df_move_pivot['Population at Start of Period']
df_move_pivot.head()

pop_move,LGA,year,Births,Deaths,Natural change,Net Migration (all sources),Population at End of Period,Population at Start of Period,pop_delta
0,Albury,2016-2021,3390.0,2219.0,1171.0,1031.0,54374.0,52171.0,2203.0
1,Armidale Regional,2016-2021,1768.0,1266.0,502.0,1921.0,32736.0,30313.0,2423.0
2,Ballina,2016-2021,1790.0,2491.0,-701.0,1945.0,44237.0,42993.0,1244.0
3,Balranald,2016-2021,194.0,96.0,98.0,8.0,2437.0,2330.0,107.0
4,Bathurst Regional,2016-2021,2500.0,1710.0,790.0,1277.0,44310.0,42244.0,2066.0


#### Population Age
- Are age brackets reasonable?
- Check dataframe style

In [8]:
popfile = "Files/Population/2019 NSW Population Projections ASGS 2019 LGA.xlsx"
df_age = pd.read_excel(popfile,sheet_name='LGA Sex Age projections',header=5,usecols="A:E",skipfooter=3)
df_age.columns=['LGA','sex','age','2016','2021']
df_age['age_delta'] = df_age['2021'] - df_age['2016']
df_age['LGA'] = df_age.LGA.str.split('(').str.get(0)

df_age_pivot = pd.pivot_table(df_age,index=['LGA','age'], values=['2016','2021','age_delta'], 
               aggfunc=({'2016':np.sum, '2021':np.sum, 'age_delta':np.sum})).reset_index()
df_age_pivot.head()

,LGA,age,2016,2021,age_delta
0,Albury,00-04,3505,3401,-104
1,Albury,05-09,3279,3510,231
2,Albury,10-14,3228,3370,142
3,Albury,15-19,3381,3306,-75
4,Albury,20-24,3744,3448,-296


In [10]:
#clusters

Child = df_age_pivot.age.unique()[:3]
Youth = df_age_pivot.age.unique()[3:5]
Adult = df_age_pivot.age.unique()[5:9]
MiddleAge = df_age_pivot.age.unique()[9:13]
Senior = df_age_pivot.age.unique()[13:]

print('Child',Child)
print('Youth',Youth)
print('Adult',Adult)
print('MiddleAge',MiddleAge)
print('Senior',Senior)

Child ['00-04' '05-09' '10-14']
Youth ['15-19' '20-24']
Adult ['25-29' '30-34' '35-39' '40-44']
MiddleAge ['45-49' '50-54' '55-59' '60-64']
Senior ['65-69' '70-74' '75-79' '80-84' '85+']


In [11]:
age_categ = [df_age_pivot['age'].isin(Child),
             df_age_pivot['age'].isin(Youth),
             df_age_pivot['age'].isin(Adult),
             df_age_pivot['age'].isin(MiddleAge),
             df_age_pivot['age'].isin(Senior)]
age_output = ['Child','Youth','Adult','MiddleAge','Senior']

df_age_pivot['age_bracket'] = np.select(age_categ,age_output)
df_age_pivot.head(20)

,LGA,age,2016,2021,age_delta,age_bracket
0,Albury,00-04,3505,3401,-104,Child
1,Albury,05-09,3279,3510,231,Child
2,Albury,10-14,3228,3370,142,Child
3,Albury,15-19,3381,3306,-75,Youth
4,Albury,20-24,3744,3448,-296,Youth
5,Albury,25-29,3485,3505,20,Adult
6,Albury,30-34,3400,3543,143,Adult
7,Albury,35-39,3143,3526,383,Adult
8,Albury,40-44,3206,3145,-61,Adult
9,Albury,45-49,3330,3284,-46,MiddleAge


### 4. Construction
Please paste cleaning codes below

In [3]:
# --read file, --rename columns
construction_file = "Files/Construction/Quarterly, Building construction prices rose, due to Homebuilder grants and government infrastructure investment.xlsx"
df_cons = pd.read_excel(construction_file,header=1,usecols="A:B", skipfooter=2)
df_cons.columns=['date','constr_index']

# --convert to datetime
df_cons['date'] = pd.to_datetime(df_cons['date'],format='%b-%y')

# --get year and quarter, --concatenate as time_period format, --drop other columns
df_cons['year'] = df_cons.date.dt.year
df_cons['quarter'] = df_cons.date.dt.quarter
df_cons['time_period'] = df_cons.year.map(str) + " Q" + df_cons.quarter.map(str)
df_cons_clean = df_cons.drop(columns=['date','year','quarter'],axis=1)
df_cons_clean.head()

,constr_index,time_period
0,100.1,2012 Q2
1,100.3,2012 Q3
2,100.2,2012 Q4
3,101.0,2013 Q1
4,101.6,2013 Q2


### 5. Weekly Income

In [2]:
# Read data in to the raw da
census_INCP = "Files/Census/POA (UR) by INCP Toal Personal Income (Weekly).csv"

incp_raw = pd.read_csv(census_INCP, skiprows=9, nrows=11142,
                       usecols=['POA (UR)', 'INCP Total Personal Income (weekly)', 'Count'])

# Rename column for easier referencing
incp_cols = {'POA (UR)':'postcode', 'INCP Total Personal Income (weekly)':'INCP_WK'}
incp_raw.rename(columns=incp_cols, inplace=True)

# Unstack
incp = incp_raw.groupby(['postcode','INCP_WK'])['Count'].sum().unstack()

# Remove the last row (grand total)
incp = incp[:-1]

incp.head(2)

INCP_WK,"$1,000-$1,249 ($52,000-$64,999)","$1,250-$1,499 ($65,000-$77,999)","$1,500-$1,749 ($78,000-$90,999)","$1,750-$1,999 ($91,000-$103,999)","$1-$149 ($1-$7,799)","$150-$299 ($7,800-$15,599)","$2,000-$2,999 ($104,000-$155,999)","$3,000 or more ($156,000 or more)","$300-$399 ($15,600-$20,799)","$400-$499 ($20,800-$25,999)","$500-$649 ($26,000-$33,799)","$650-$799 ($33,800-$41,599)","$800-$999 ($41,600-$51,999)",Negative income,Nil income,Not applicable,Not stated,Total
postcode,,,,,,,,,,,,,,,,,,
"2000, NSW",1676,1028,977,677,502,1112,1456,1796,1978,2201,1915,1599,1638,204,3297,1251,4115,27411
"2006, NSW",11,9,9,3,275,291,4,11,98,47,31,5,3,9,267,15,156,1261


In [3]:
# Remove 'NSW' in the index and cast postcode to int64
incp.reset_index(inplace=True)
incp['postcode'] = incp['postcode'].str.split(",", n=1, expand=True)
incp['postcode'] = incp['postcode'].astype('int64')
incp = incp.set_index('postcode')

In [4]:
# Clean column names
income_cols= {'$1,000-$1,249 ($52,000-$64,999)' : '$1000-1249', 
            '$1,250-$1,499 ($65,000-$77,999)' : '$1250-1499',
            '$1,500-$1,749 ($78,000-$90,999)' : '$1500-1749 ', 
            '$1,750-$1,999 ($91,000-$103,999)': '$1750-1999',
            '$1-$149 ($1-$7,799)': '$1-149', 
            '$150-$299 ($7,800-$15,599)' : '$150-299',
            '$2,000-$2,999 ($104,000-$155,999)':'$2000-2999',
            '$3,000 or more ($156,000 or more)':'>=$3000', 
            '$300-$399 ($15,600-$20,799)':'$300-399',
            '$400-$499 ($20,800-$25,999)':'$400-499', 
            '$500-$649 ($26,000-$33,799)':'$500-649',
            '$650-$799 ($33,800-$41,599)':'$650-799', 
            '$800-$999 ($41,600-$51,999)':'$800-999'}
incp.rename(columns=income_cols, inplace=True)

# Combine 'not applicable' and 'not stated' into 'total_na'
incp['total_na'] = incp['Not applicable'] + incp['Not stated']

# Drop the 'Total column'
incp = incp.drop(columns=['Not applicable', 'Not stated', 'Total'], axis=1)

# Reorder columns
cols = incp.columns.tolist()
cols = ['$1-149','$150-299','$300-399','$400-499','$500-649','$650-799',
        '$800-999','$1000-1249','$1250-1499','$1500-1749 ',
        '$1750-1999','$2000-2999','>=$3000',
        'Negative income','Nil income','total_na']
incp=incp[cols]

incp.head(1)

INCP_WK,$1-149,$150-299,$300-399,$400-499,$500-649,$650-799,$800-999,$1000-1249,$1250-1499,$1500-1749,$1750-1999,$2000-2999,>=$3000,Negative income,Nil income,total_na
postcode,,,,,,,,,,,,,,,,
2000,502,1112,1978,2201,1915,1599,1638,1676,1028,977,677,1456,1796,204,3297,5366


In [5]:
# Create income buckets and save into incp_gr
incp['INCP_LOW'] = incp.iloc[:, 0:6].sum(axis=1)
incp['INCP_MID'] = incp.iloc[:, 6:10].sum(axis=1)
incp['INCP_HIGH'] = incp.iloc[:, 10:13].sum(axis=1)
incp['INCP_NEG_NIL'] = incp.iloc[:, 13:15].sum(axis=1)
incp_gr = incp[['INCP_LOW', 'INCP_MID', 'INCP_HIGH', 'INCP_NEG_NIL']]

# Reset index
incp_gr.reset_index(inplace=True)

incp_gr.head(1)

INCP_WK,postcode,INCP_LOW,INCP_MID,INCP_HIGH,INCP_NEG_NIL
0,2000,9307,5319,3929,3501


*The resulting cleanead df is <b>incp_gr</b>*

----

### 6. Household size

In [6]:
# Read data
census_cprf = "Files/Census/POA by CPRF Count of Persons in Family by STATE.xlsx"
cprf = pd.read_excel(census_cprf, sheet_name="Data Sheet 0", skiprows=9, nrows=619)

# Remove redundant rows and columns 
cprf = cprf[1:] #remove the first row
cprf = cprf.drop(columns='CPRF Count of Persons in Family') # remove the first column

# Rename columns
cprf_cols= {'Unnamed: 1' : 'postcode', 
            'Two persons in family' : 'CPRF_2',
            'Three persons in family' : 'CPRF_3', 
            'Four persons in family': 'CPRF_4',
            'Five persons in family': 'CPRF_5', 
            'Six or more persons in family' : 'CPRF_6+',
            'Not applicable':'CPRF_na',
            'Total' :'CPRF_TOTAL_FAM_NO'}
cprf.rename(columns=cprf_cols, inplace=True)

cprf.head(1)

,postcode,CPRF_2,CPRF_3,CPRF_4,CPRF_5,CPRF_6+,CPRF_na,CPRF_TOTAL_FAM_NO
1,"2000, NSW",3453.0,857.0,354.0,54.0,21.0,8125.0,12861.0


In [7]:
# Remove 'NSW' in the index and cast postcode to int64
cprf.reset_index(inplace=True)
cprf['postcode'] = cprf['postcode'].str.split(",", n=1, expand=True)
cprf['postcode'] = cprf['postcode'].astype('int64')
cprf = cprf.set_index('postcode')
cprf = cprf.drop(columns='index', axis=1)

In [8]:
# Reset index for merging
cprf.reset_index(inplace=True)
cprf.head(1)

,postcode,CPRF_2,CPRF_3,CPRF_4,CPRF_5,CPRF_6+,CPRF_na,CPRF_TOTAL_FAM_NO
0,2000,3453.0,857.0,354.0,54.0,21.0,8125.0,12861.0


*The resulting cleanead df is <b>cprf</b>*

----

### 7. Additional Feature 1 
Please paste cleaning codes below

### 8. Additional Feature 2
Please paste cleaning codes below

### 9. Additional Feature 3
Please paste cleaning codes below

## USE BELOW CELL TO MERGE FEATURES INTO THE MASTER DF, IGNORE FOR NOW

--------